In [1]:
pip install contractions

     |████████████████████████████████| 327kB 9.4MB/s 
     |████████████████████████████████| 286kB 31.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85415 sha256=cd0bb5c25f556a00ae13e55fa5e88f1d6f03363040682760258995e07aad7e9d
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Binary_Logistic_Regression.py to Binary_Logistic_Regression.py
User uploaded file "Binary_Logistic_Regression.py" with length 4166 bytes


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Binary_Logistic_Regression import BinaryLogisticRegression
from nltk.tokenize import word_tokenize
import re
import nltk
from contractions import contractions_dict
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import time
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
data = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/Email Classification using Logistic Regression/emails.csv")

In [8]:
def pre_processing(text):
    regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'
    en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

    #strip titles
    if "Subject: re :" in text:
        text = text[13:]
    elif "Subject: news :" in text:
        text = text[15:]
    else:
        text = text[8:]

    #tokenize using nltk
    text = word_tokenize(text)

    #normalize tokens
    text = [token.lower() for token in text]

    #contraction expansion
    text = [contractions_dict[token] if token in contractions_dict.keys() else token for token in text]

    #remove waste words
    l = list()
    for token in text:
      if not(re.search(regex,token)):
        l.append(token)
    text = l
    del l

    #split
    text = [re.split(regex,token)[0] for token in text]

    #remove stop words
    l = list()
    for token in text:
      if not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token)):
        l.append(token)
    text = l
    del l

    #assigning pos tag to each token
    text = pos_tag(text)
    text = [(i[0],wordnet.ADJ) if i[1][0] == 'J' else (i[0],wordnet.VERB) if i[1][0] == 'V' else (i[0],wordnet.ADV) if i[1][0] == 'R' else (i[0],wordnet.NOUN) for i in text]

    #applying lemmatizer
    lemmatizer = WordNetLemmatizer()
    if len(text) > 0:
      text = ['' if token == None else lemmatizer.lemmatize(word=token[0],pos=token[1]) for token in text]

    del regex, en_stop_words, lemmatizer
    return text

In [9]:
# data preprocessing
data['text'] = data['text'].apply(lambda x: pre_processing(x))

In [10]:
data

,text,spam
0,"[naturally, irresistible, corporate, identity,...",1
1,"[stock, trading, gunslinger, fanny, merrill, m...",1
2,"[unbelievable, new, home, easy, im, want, home...",1
3,"[color, print, special, request, additional, i...",1
4,"[money, software, cd, software, compatibility,...",1
...,...,...
5723,"[research, development, charge, gpg, forward, ...",0
5724,"[receipt, visit, jim, thanks, invitation, visi...",0
5725,"[enron, case, study, update, wow, day, super, ...",0
5726,"[interest, david, shirley, crenshaw, assistant...",0


In [11]:
preprocessed_data = data.copy()

vocab = set()
for list_of_tokens in preprocessed_data['text']:
  vocab = vocab.union(set(list_of_tokens))

vocab_dict = dict(zip(vocab,list(range(0,len(vocab)))))

preprocessed_data['text'] = preprocessed_data['text'].apply(lambda x: " ".join(x))

corpus = [i for i in preprocessed_data['text']]

vectorizer = TfidfVectorizer(vocabulary=vocab_dict)
tf_idf_matrix = vectorizer.fit_transform(corpus).toarray()

del preprocessed_data, vocab, vocab_dict, corpus, vectorizer

print(tf_idf_matrix.shape)

(5728, 29610)


In [12]:
class Email_classification(BinaryLogisticRegression):
  def __init__(self, tf_idf_matrix, labels, n_comp, LR=10**(-6), ET=10**(-5)):

    pca = PCA(n_components=n_comp)
    self.tf_idf_matrix_reduced = pca.fit_transform(tf_idf_matrix)

    self.reduced_data = pd.DataFrame(self.tf_idf_matrix_reduced)
    self.reduced_data['spam'] = labels


    spam_data = self.reduced_data[self.reduced_data['spam'] == 1]
    not_spam_data = self.reduced_data[self.reduced_data['spam'] == 0]

    self.training = pd.concat([spam_data.iloc[:(int(self.reduced_data.shape[0]*0.7)//2),:], not_spam_data.iloc[:(int(self.reduced_data.shape[0]*0.7)//2),:]], axis=0)

    remaining = pd.concat([spam_data.iloc[(int(self.reduced_data.shape[0]*0.7)//2):,:], not_spam_data.iloc[(int(self.reduced_data.shape[0]*0.7)//2):,:]], axis=0)

    X_cv, X_test, Y_cv, Y_test = train_test_split(remaining.iloc[:,:-1], remaining['spam'], test_size=1/3)

    self.cv = pd.DataFrame(X_cv)
    self.cv['spam'] = Y_cv

    self.testing = pd.DataFrame(X_test)
    self.testing['spam'] = Y_test

    super().__init__(1, LearningRate=LR, ErrorTolerance=ET)

    del X_test, Y_test, X_cv, Y_cv, remaining, n_comp, pca, tf_idf_matrix, labels

In [13]:
def evaluate(predicted, actual):
    TP = np.count_nonzero((predicted == 1) & (actual == 1))
    TN = np.count_nonzero((predicted == 0) & (actual == 0))
    FP = np.count_nonzero((predicted == 1) & (actual == 0))
    FN = np.count_nonzero((predicted == 0) & (actual == 1))
    
    if (TP + TN + FP + FN) == 0:
        accuracy = 0
    else:
        accuracy = (TP + TN)/(TP + TN + FP + FN)
    
    if (TP + FP) == 0:
        precision = 0
    else:
        precision = TP/(TP + FP)
    
    if (TP + FN) == 0:
        recall = 0
    else:
        recall = TP/(TP + FN)
        
    if (precision + recall) == 0:
        f1_score = 0
    else:
        f1_score = (2 * precision * recall)/(precision + recall)
    
    return (accuracy, precision, recall, f1_score)

In [14]:
Results = dict()
best = {'Accuracy':0, 'Precision':0, 'Recall':0, 'F1 Score':0, 'Best Obj':'obj', 'Parameter':()}

# Trying Different Combinaitions of Hyperparameters

In [ ]:
for n_comp in np.arange(2,1241,20):
    for LR in [6,7,8,9,10]:
        for ET in [5,6,7,8,9,10]:
            obj = Email_classification(tf_idf_matrix, data['spam'], n_comp, LR=10**(-LR), ET=10**(-ET))
            obj.fit(obj.training.iloc[:,:-1], obj.training['spam'])
            print('---------------------------------------------------------------------------')
            obj.predict(obj.cv.iloc[:,:-1])
            Results[(n_comp, f'10**(-{LR})', f'10**(-{ET})')] = evaluate(obj.predicted_labels, obj.cv['spam'])
            if Results[(n_comp, f'10**(-{LR})', f'10**(-{ET})')][0] > best['Accuracy']:
                best['Accuracy'] = Results[(n_comp, f'10**(-{LR})', f'10**(-{ET})')][0]
                best['Precision'] = Results[(n_comp, f'10**(-{LR})', f'10**(-{ET})')][1]
                best['Recall'] = Results[(n_comp, f'10**(-{LR})', f'10**(-{ET})')][2]
                best['F1 Score'] = Results[(n_comp, f'10**(-{LR})', f'10**(-{ET})')][3]
                best['Best Obj'] = obj
                best['Parameter'] = (n_comp, f'10**(-{LR})', f'10**(-{ET})')
            else:
                del obj

In [ ]:
best

# From over 2232 Hyperparameter combinaitons we find our best model

In [ ]:
best

In [ ]:
best_obj = best['Best obj']

In [ ]:
best_obj.fit(best_obj.testing)

In [ ]:
print(evaluate(best_obj.predicted_labels, best_obj.testing['spam']))